In [1]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import scipy.signal
import random
from sklearn.cluster import KMeans

test_data_file = {}

def select_test_files(num_testfiles):
        for i in range(0 , num_testfiles):                  # we need to take 10% of the audio files for the training set
            rand_spkr = random.randint(1,5)     # choose a random speaker 1-10 
            rootdir = 'C:\\Users\\James Hoffman\\Desktop\\Speech_Recognition\\test_files\\' + str(rand_spkr)
            x = 0                                # simple counter 
            random_file = random.randint(0,500)  # 0-500 because of the 500 audio files per folder
            for filename in os.listdir(rootdir): 
                if(x == random_file):
                    test_data_file[filename] = rand_spkr  # adds these random test files 
                x += 1

def display_graph_for_data(data , sampling_rate):
    plt.figure(figsize=(12 , 4))
    librosa.display.waveplot(data, sr=sampling_rate)
    plt.show()

'''
     takes the location of an audio file and does the fourier transform,
     flattens that and returns it as a single vector
'''    
def transform(audio):
    # sr == sampling rate
    data, sampling_rate = librosa.load(audio, sr=4000)
    spectral = 10 * np.log10(np.abs(librosa.core.stft(data, n_fft=512, hop_length=80, window=scipy.signal.hanning)))
    return spectral


'''
     takes a single directory (one of the numbered one from like 1-30)
     and returns an np.array which contains the flattened data that can
     be fed into the kmeans algorithm
'''
def parse_directory(audio_directory , dir_num, trainfiles): 
    
    flat_audio = np.empty([0,0])         # this will be reassigned anyway and returned
    first_a = True                       # this is for starting the list of matrices
    x = 0                                # simple counter for the purpose of testing
    
    # this just iterates through the directory with all the folders in it
    for filename in os.listdir(audio_directory):
        
        # we don't want to use any of the test data, so if the file is in the test data then we just skip it 
        if filename in test_data_file.keys():
            if(dir_num == test_data_file[filename]):
                continue
        
#         this is because doing all 500 would take a really ong time
        x += 1
        if(x > trainfiles):
            break
            
        # Each file is sent to the transform() function and has a matrix returned
        # All these results are placed next to eachother in a list to be sent to the Kmeans value
        if(first_a == True):
            flat_audio = transform(audio_directory + "\\" + filename)
            first_a = False
        else:
            append_audio = transform(audio_directory + "\\" + filename)
            flat_audio = np.hstack((flat_audio, append_audio))
            
    # we are returned a single list of transformed matrices that had the fourier transform done on them
    return flat_audio


'''
     takes a single directory (the one with all the data)
     and returns an np.array which has the all the np arrays that contain the
     vector for each of the numbered speakers in that directory
'''
def iterate_test_folders(rootdir, trainfiles):
    audio_vectors = []
    for subdir, dirs, files in os.walk(rootdir):
        for dir in dirs:
            print("iterating through directory: " , dir)
            p_d_ = parse_directory(rootdir + dir , dir, trainfiles)
            audio_vectors.append(p_d_)
            
    return audio_vectors

    

In [2]:
def get_vectors(trainfiles):
    rootdir = 'C:\\Users\\James Hoffman\\Desktop\\Speech_Recognition\\test_files\\'
    list_of_flattened_vectors = iterate_test_folders(rootdir, trainfiles)
    
    return list_of_flattened_vectors
    

In [3]:
from sklearn import decomposition
from sklearn import datasets

def main(testfiles , trainfiles , num_clusteres):
    print("Starting...")
    select_test_files(testfiles)
    sample = get_vectors(trainfiles)
    kmeans = [0,0,0,0,0]

    
    for i in range(0,5):
        kmeans[i] = KMeans(n_clusters = num_clusteres, random_state=0).fit(sample[i].T)
    
    
    train_centers = get_centers(kmeans)
    test_transformed,speaker_nums = generate_test_transformed()
    
    correct = 0
    incorrect = 0
    
    
    for i in range(0, len(test_transformed)):         # i is the index for the test file
        min_v,index,ref = find_least_euclidean(test_transformed[i], train_centers)
#         print("Predicted: " , (index) , " :: Actual Speaker: " , speaker_nums[i])
        if(index == speaker_nums[i]):
            correct = correct + 1
        else:
            incorrect = incorrect + 1
            
    print("Correct: " , correct)
    print("Incorrect: " , incorrect)
    print("percent correct" , ((correct / (correct + incorrect)) * 100))
    

In [4]:
 def get_centers(kmeans):
        centers = []
        for i in range(0,5):
            centers.append(np.array(kmeans[i].cluster_centers_))
        return centers

In [5]:
def generate_test_kmeans_center():
    test_kmeans_centers = []
    
    # iterate through test data and gather the centers
    for file_n in test_data_file:
        folder = test_data_file[file_n]
        f_trans = transform('C:\\Users\\James Hoffman\\Desktop\\Speech_Recognition\\test_files\\'+ str(folder) +'\\' + str(file_n))
        kmeans_test = KMeans(n_clusters = 128, random_state=0).fit(f_trans)
        center = np.array(kmeans_test.cluster_centers_)
        test_kmeans_centers.append(center)
    return test_kmeans_centers

In [6]:
def generate_test_transformed():
    test = []
    speaker_nums = []
    
    # iterate through test data and gather the centers
#     print(test_data_file)
    for file_n in test_data_file:
        folder = test_data_file[file_n]
        f_trans = transform('C:\\Users\\James Hoffman\\Desktop\\Speech_Recognition\\test_files\\'+ str(folder) +'\\' + str(file_n))
        test.append(f_trans.T)
        speaker_nums.append(folder)
    return test, speaker_nums

In [7]:
from scipy.spatial import distance


def find_least_euclidean(test_transformed , train_centers):
    min_v = 10000
    index = -1
    min_i = 0
    ref = 0
    for test_row in test_transformed:
        index = 0
        for center in train_centers:
            index = index + 1
            for train_row in center:
                euclidean = distance.euclidean(test_row , train_row)
                if(euclidean < min_v):
                    min_v = euclidean
                    min_i = index
                    ref = center
                    
    return min_v,min_i,ref

In [8]:
# testfiles , trainfiles, num_clusters
main(250 , 500 , 128) # start the program

Starting...
iterating through directory:  1
iterating through directory:  2
iterating through directory:  3
iterating through directory:  4
iterating through directory:  5
Correct:  180
Incorrect:  56
percent correct 76.27118644067797
